In [1]:
import pandas as pd
import numpy as np

In [2]:
dataDir = '../../Data/advertisment/Raw/'
trainFile = 'round1_ijcai_18_train_20180301.txt'
testFile = 'round1_ijcai_18_test_a_20180301.txt'

In [3]:
dfTrain = pd.read_table(dataDir+trainFile,sep = ' ')

In [16]:
dfTrain.isnull().sum()

instance_id                  0
item_id                      0
item_category_list           0
item_property_list           0
item_brand_id                0
item_city_id                 0
item_price_level             0
item_sales_level             0
item_collected_level         0
item_pv_level                0
user_id                      0
user_gender_id               0
user_age_level               0
user_occupation_id           0
user_star_level              0
context_id                   0
context_timestamp            0
context_page_id              0
predict_category_property    0
shop_id                      0
shop_review_num_level        0
shop_review_positive_rate    0
shop_star_level              0
shop_score_service           0
shop_score_delivery          0
shop_score_description       0
is_trade                     0
item_category_list_cnt       0
item_property_list_cnt       0
dtype: int64

In [5]:
dfTrain.head(4)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0


In [8]:
dfTrain['item_property_list_cnt'] = dfTrain['item_property_list'].apply(lambda x: x.count(';'))
dfTrain['item_property_list_cnt'].unique()

array([21, 49, 45, 29, 31, 34, 32, 47, 28, 38, 24, 36, 23, 33, 26, 58, 25,
       42, 19, 20, 27, 16, 40, 12, 18, 37, 22, 44, 35, 48, 50, 30, 43, 15,
       17, 39, 14, 13, 63, 54, 41, 46, 56,  9, 11, 99, 53, 75, 51, 62,  5,
       52, 59, 70, 61, 57, 66, 55, 60, 67, 10, 87, 96, 72, 74, 84, 81, 88,
        8, 69,  7, 92, 65, 90,  6, 71, 64, 73, 98], dtype=int64)

In [10]:
tt = dfTrain['item_property_list_cnt'].unique()
tt.sort()
tt

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73,
       74, 75, 81, 84, 87, 88, 90, 92, 96, 98, 99], dtype=int64)

In [14]:
tt = dfTrain['item_price_level'].unique()
tt.sort()
tt

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 16, 17], dtype=int64)

In [20]:
dfTrain['is_trade'].mean()

0.01886693799697995